In [1]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tqdm import tqdm 

In [2]:
# Cargar el modelo de SpaCy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

class Tok2VecClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model='en_core_web_md', batch_size=1000, n_process=4):
        """
        Inicializa el clasificador.

        :param model: Nombre del modelo de SpaCy a cargar.
        :param batch_size: Tamaño de los lotes para nlp.pipe.
        :param n_process: Número de procesos a utilizar en nlp.pipe.
        """
        self.model_name = model
        self.batch_size = batch_size
        self.n_process = n_process
        self.nlp = spacy.load(self.model_name, disable=['parser', 'ner'])
        self.logistic_model = LogisticRegression(max_iter=1000, n_jobs=self.n_process)
    
    def _transform_texts(self, X):
        """
        Transforma una lista de textos en vectores utilizando nlp.pipe.

        :param X: Lista o Serie de textos.
        :return: Array de vectores.
        """
        vectors = []
        # nlp.pipe ya maneja el progreso de procesamiento
        for doc in tqdm(
            self.nlp.pipe(X, batch_size=self.batch_size, n_process=self.n_process, disable=['parser', 'ner']),
            total=len(X),
            desc="Transformando textos"
        ):
            vectors.append(doc.vector)
        return np.array(vectors)
    
    def fit(self, X, y):
        """
        Ajusta el modelo de regresión logística utilizando vectores de texto.

        :param X: Lista o Serie de textos de entrenamiento.
        :param y: Etiquetas de entrenamiento.
        :return: El clasificador ajustado.
        """
        X_vec = self._transform_texts(X)
        self.logistic_model.fit(X_vec, y)
        return self
    
    def predict(self, X):
        """
        Realiza predicciones sobre nuevos datos de texto.

        :param X: Lista o Serie de textos a predecir.
        :return: Array de predicciones.
        """
        X_vec = self._transform_texts(X)
        return self.logistic_model.predict(X_vec)
    
    def predict_proba(self, X):
        """
        Devuelve las probabilidades de las predicciones.

        :param X: Lista o Serie de textos a predecir.
        :return: Array de probabilidades.
        """
        X_vec = self._transform_texts(X)
        return self.logistic_model.predict_proba(X_vec)

In [3]:
# Cargar el dataset
file_path = r"C:\Users\rafae\MisCosas\IMDB\IMDB-Kaggle\IMDB Dataset.csv"
data = pd.read_csv(file_path)

In [4]:
# Preprocesar el texto
def preprocess_texts(texts, nlp, batch_size=1000, n_process=4):
    cleaned_reviews = []
    # Utilizar nlp.pipe con múltiples procesos
    for doc in tqdm(nlp.pipe(texts, batch_size=batch_size, n_process=n_process), total=len(texts)):
        tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
        cleaned_reviews.append(' '.join(tokens))
    return cleaned_reviews

# Preprocesar el conjunto de datos
data['cleaned_review'] = preprocess_texts(data['review'], nlp, batch_size=1000, n_process=4)


100%|██████████| 50000/50000 [03:05<00:00, 268.89it/s]


In [5]:
# Separar el texto y las etiquetas
X = data['cleaned_review']
y = data['sentiment']

# Dividir el dataset en entrenamiento (70%), validación (20%) y test (10%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.22, random_state=42, stratify=y_temp)


In [6]:
# Crear el pipeline de clasificación con Tok2Vec
tok2vec_classifier = Tok2VecClassifier(model='en_core_web_lg', batch_size=1000, n_process=6)
tok2vec_classifier.fit(X_train, y_train)

Transformando textos: 100%|██████████| 35100/35100 [01:07<00:00, 519.39it/s] 


AttributeError: 'Tok2VecClassifier' object has no attribute 'model'

AttributeError: 'Tok2VecClassifier' object has no attribute 'model'

AttributeError: 'Tok2VecClassifier' object has no attribute 'model'

In [7]:
# Evaluar el modelo en el conjunto de validación
y_val_pred = tok2vec_classifier.predict(X_val)

# Evaluar el modelo en el conjunto de test
y_test_pred = tok2vec_classifier.predict(X_test)

Transformando textos: 100%|██████████| 5000/5000 [00:31<00:00, 158.84it/s] 


In [8]:
# Mostrar el rendimiento
print("Reporte de clasificación (validación):")
print(classification_report(y_val, y_val_pred))
print("Precisión (validación):", accuracy_score(y_val, y_val_pred))

print("Reporte de clasificación (test):")
print(classification_report(y_test, y_test_pred))
print("Precisión (test):", accuracy_score(y_test, y_test_pred))

Reporte de clasificación (validación):
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      4950
    positive       0.85      0.85      0.85      4950

    accuracy                           0.85      9900
   macro avg       0.85      0.85      0.85      9900
weighted avg       0.85      0.85      0.85      9900

Precisión (validación): 0.8521212121212122
Reporte de clasificación (test):
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84      2500
    positive       0.84      0.85      0.84      2500

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000

Precisión (test): 0.8426
